In [1]:
import os
import glob
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

In [2]:
features = pd.DataFrame()

In [ ]:
'''alt_max = []
alt_av = []
dist_av = []
plane_speed_av = []
dalt_av = []
fnames = []'''

In [3]:
f_list = []
path = './data/'
for filename in glob.glob(os.path.join(path, '*.tsv')):
    if 'unknown' in filename:
        continue
    df = pd.read_csv(filename, sep = '\t')
    dphi = np.diff(np.asarray(df['latitude'].values, dtype = np.float64))
    dlambda = np.diff(np.asarray(df['longitude'].values, dtype = np.float64))
    dalt = np.diff(np.diff(np.asarray(df['altitude_meters'].values, dtype = np.float64)))
    dt = np.diff(np.asarray(df['timestamp'].values, dtype = np.float64))
    if 0 in dt:
        continue
    if len(dt) < 20:
        continue
    try:
        dphi /= dt
        dlambda /= dt
        dalt /= dt[1:]
    except:
        continue
    
    dalt_mean = 0
    if np.where(np.absolute(dalt) > 0.5):
        dalt_mean = np.mean(dalt[np.where(np.absolute(dalt) > 0.5)])
    plane_speed = np.sqrt(np.power(dphi, 2) + np.power(dlambda, 2))
    dist = ((df['latitude'].values[-1] - df['latitude'].values[0]) ** 2 +
        (df['longitude'].values[-1] - df['longitude'].values[0]) ** 2)
    dist = np.sqrt(dist)
    if np.isnan(np.sum(plane_speed)):
        continue
    if np.isnan(np.sum(dalt)):
        continue
    if dist == 0:
        continue
    l = np.asarray([np.max(np.asarray(df['altitude_meters'].values, 
            dtype = np.float64)), dalt_mean,
            np.mean(plane_speed), dist])
    is_nan = False
    for i in np.isnan(l):
        is_nan = i and is_nan
    if not is_nan:
        f_list.append([filename, np.max(np.asarray(df['altitude_meters'].values, 
            dtype = np.float64)), dalt_mean,
            np.mean(plane_speed), dist])

/home/sriram12/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/sriram12/.local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
len(f_list)

In [ ]:
f_list = np.asarray(f_list)

In [ ]:
len(f_list[5])

In [ ]:
for idx, x in enumerate(['fname', 'alt_max', 'vert_acc', 'horiz_speed', 'dist']):
    features[x] = f_list[:, idx]

In [ ]:
features.to_csv('features.csv', index = False)

In [ ]:
features.describe()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
features['alt_max'].mean()

In [ ]:
scaled_data = scaler.fit_transform(features.drop(['fname'], axis = 1).values)

In [ ]:
features.drop(['fname'], axis = 1).to_csv('features_only.csv', index = False)